In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


## modelagem GLM

In [2]:
%%time
df_mod_mp_feb23jul24_v2 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23jul24_v2.csv",
                     encoding='latin-1',
                     delimiter='|')


df_mod_mp_feb23jul24_v2.shape

CPU times: total: 13.9 s
Wall time: 17.2 s


(575924, 105)

In [3]:
premio_gbm = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_julho_gbm_python_v2.csv",
                     encoding='latin-1',
                     delimiter='|')
premio_gbm.shape

(575924, 3)

In [4]:
df_mod_mp_feb23jul24_v2 = pd.merge(df_mod_mp_feb23jul24_v2,premio_gbm[['Chave','GBM_mercado']],on='Chave', how='left')

In [5]:
df_mod_mp_feb23jul24_v2['Grupo_veiculo_new']=df_mod_mp_feb23jul24_v2['novo']

In [6]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(df_mod_mp_feb23jul24_v2)

Your selected dataframe has 107 columns and 575924 Rows.
There are 16 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
perf_cond_menor26,0,391245,67.9,391245,67.9,object
perf_est_civil,0,227357,39.5,227357,39.5,object
Corr_Classif_5_hibr,0,99433,17.3,99433,17.3,object
perf_modalidade,0,40243,7.0,40243,7.0,object
novo,0,1622,0.3,1622,0.3,object
Grupo_veiculo_new,0,1622,0.3,1622,0.3,object
per_com,652,1372,0.2,2024,0.4,float64
MD_est_civil,0,566,0.1,566,0.1,object
FABRICA,0,378,0.1,378,0.1,object
cod_nacionalidade,0,378,0.1,378,0.1,float64


#### prepara base de modelagem glm

In [7]:
df_mod_filtered=df_mod_mp_feb23jul24_v2[["premio_ofertado","premio_total", 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod','GBM_mercado']]

df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]
df_mod_filtered['GBM_mercado']  = df_mod_filtered['GBM_mercado']/(1-df_mod_filtered['per_com']/100)


In [8]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['FABRICA'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
#df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]

df_mod_filtered = df_mod_filtered[df_mod_filtered['anomes_dt_ter'].isin([202405,202406,202407])]


df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('premio_ofertado >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total >=0.7)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]

df_mod_filtered.shape

(80201, 51)

In [9]:
df_mod=df_mod_filtered.copy()

In [10]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [11]:
df_mod = categorizar(df_mod)

In [12]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(64070, 65)

In [13]:
df_train['Premio_IC_100']=df_train['Premio_IC100']

In [109]:
df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC_100','premio_anterior','GBM_mercado']]
df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_y = df_train[['ind_renovacao']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_cat_y,df_num, df_cat_glm],axis=1)

In [110]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula =  'ind_renovacao ~ 1 + np.log(premio) +  np.divide(premio,is_casco_ofer) + np.log(np.divide(premio, GBM_mercado)) +  np.log(np.divide(premio, Premio_IC_100))+ np.log((np.divide(premio, premio_anterior))) + np.log(MD_tempo_casa_2+1) + per_com + per_com_bin_p_1_10 + canal_bin_Corretor_Mais + Cod_End_bin_13509 + fator_ajuste_bin_f_101_120 + classe_bonus_bin_00 + UF_bin_AL_CE_RS + UF_bin_AP_DF_ES_PA_PB_RN_SC + UF_bin_GO_MA_MG + UF_bin_MT_PR + UF_bin_RJ + MD_comb_bin_Gasolina + md_tipo_renov_ant_bin_1_SeguroNovo + md_tipo_renov_ant_bin_3_RenovCongenere + fx_ir_12m_bin_01_00_20 + fx_ir_12m_bin_03_40_60 + fx_ir_12m_bin_04_60_80 + fx_ir_12m_bin_05_80_100 + MD_tipo_renov_bin_3_RenovPropria_com_sin + idade_ult_RNS_bin_01_05_anos + idade_ult_RNS_bin_06_inf ' 
#formula = 'ind_renovacao ~ 1 + np.log(premio) + np.log(np.divide(premio, premio_anterior)) + np.where(np.divide(premio, premio_anterior) > 1, 1, 0)'
model_glm = smf.logit(formula , data=df_cat_glm).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.541466
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:                64070
Model:                          Logit   Df Residuals:                    64042
Method:                           MLE   Df Model:                           27
Date:                Tue, 03 Sep 2024   Pseudo R-squ.:                  0.2188
Time:                        13:38:52   Log-Likelihood:                -34692.
converged:                       True   LL-Null:                       -44409.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        1.3356      0.246      5.426      0.000       0.853       1.818
np.log(premio)                                  -0.2576      0.029     -8.746      0.000      -0.315      -0.200
np.divide(premio, is_casco_ofer)                 1.3647      0.562      2.428      0.015       0.263       2.466
np.log(np.divide(premio, GBM_mercado))          -0.8782      0.059    -14.789      0.000      -0.995      -0.762
np.log(np.divide(premio, Premio_IC_100))        -0.7950      0.042    -18.863      0.000      -0.878      -0.712
np.log((np.divide(premio, premio_anterior)))    -1.6731      0.054    -31.182      0.000      -1.778      -1.568
np.log(MD_tempo_casa_2 + 1)                      0.3797      0.032     11.879      0.000       0.317       0.442
per_com                                          0.0011      0.002      0.570      0.569      -0.003       0.005
per_com_bin_p_1_10                               0.1253      0.052      2.390      0.017       0.023       0.228
canal_bin_Corretor_Mais                          0.1334      0.024      5.537      0.000       0.086       0.181
Cod_End_bin_13509                               -0.1382      0.030     -4.624      0.000      -0.197      -0.080
fator_ajuste_bin_f_101_120                       0.3919      0.075      5.191      0.000       0.244       0.540
classe_bonus_bin_00                             -0.1397      0.084     -1.659      0.097      -0.305       0.025
UF_bin_AL_CE_RS                                 -0.2305      0.030     -7.626      0.000      -0.290      -0.171
UF_bin_AP_DF_ES_PA_PB_RN_SC                     -0.2832      0.031     -9.171      0.000      -0.344      -0.223
UF_bin_GO_MA_MG                                  0.0024      0.030      0.078      0.938      -0.057       0.062
UF_bin_MT_PR                                    -0.1476      0.034     -4.303      0.000      -0.215      -0.080
UF_bin_RJ                                        0.2817      0.050      5.588      0.000       0.183       0.381
MD_comb_bin_Gasolina                             0.3336      0.045      7.341      0.000       0.245       0.423
md_tipo_renov_ant_bin_1_SeguroNovo              -0.4115      0.040    -10.270      0.000      -0.490      -0.333
md_tipo_renov_ant_bin_3_RenovCongenere           0.0127      0.033      0.382      0.702      -0.052       0.078
fx_ir_12m_bin_01_00_20                          -0.7300      0.037    -19.702      0.000      -0.803      -0.657
fx_ir_12m_bin_03_40_60                           0.4343      0.023     19.173      0.000       0.390       0.479
fx_ir_12m_bin_04_60_80                           1.1066      0.031     35.877      0.000       1.046       1.167
fx_ir_12m_bin_05_80_100                          1.9561      0.051     38.237      0.000       1.856       2.056
MD_tipo_renov_bin_3_RenovPropria_com_sin        -0.0742      0.047     -1.585

In [16]:
 model_glm.params['np.log((np.divide(premio, premio_anterior)))']

-1.673076119185243

In [18]:
X_vals

array([46.93826502, 56.57798247, 68.58974826, 81.47930721, 85.00868992,
       35.24210809, 80.3181335 , 16.06098619, 49.42396792, 87.82231562,
       41.24344485, 93.76237643, 60.59325961, 54.28901514, 41.51942922,
       44.04931368, 84.23551992, 17.51366067, 95.46878583, 60.24622995,
       45.60096626, 93.99966627, 59.49986118, 89.30570346, 73.51596473,
       12.07187614])

In [21]:
indicadora

array([[4.94910000e+04, 2.60071567e+03, 2.56989446e+03, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.05168000e+05, 8.03149777e+03, 1.12766090e+04, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.15632000e+05, 3.82558036e+03, 4.36304867e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.08514000e+05, 3.72513823e+03, 2.13456284e+03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.70606000e+05, 6.77621705e+03, 6.28688802e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.49688000e+05, 4.24062082e+03, 2.09024865e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [111]:
df = df_cat_glm

In [113]:
df['tarifa']=df['premio']
df['numbers'] = df['premio']/df['tarifa']
if df['numbers'].any()>1 :
    df['X27'] = numbers-1
else:
     df['X27']=1

In [114]:
df

,ind_renovacao,premio,per_com,MD_tempo_casa_2,is_casco_ofer,Premio_IC_100,premio_anterior,GBM_mercado,canal_bin_Corretor_Mais,canal_bin_Demais,...,fx_ir_12m_bin_04_60_80,fx_ir_12m_bin_05_80_100,MD_tipo_renov_bin_2_RenovPropria_sem_sin,MD_tipo_renov_bin_3_RenovPropria_com_sin,idade_ult_RNS_bin_01_05_anos,idade_ult_RNS_bin_06_inf,idade_ult_RNS_bin_Sem_Sinistro,tarifa,numbers,X27
8,0,2487.76,14.0,3,49491.0,2569.894455,1426.222541,2600.715672,0,1,...,0,0,0,1,1,0,0,2487.76,1.0,1
11,0,11826.61,18.0,1,305168.0,11276.608968,12195.826776,8031.497770,0,1,...,0,0,1,0,0,1,0,11826.61,1.0,1
12,0,5098.39,2.0,1,115632.0,4363.048672,4703.393989,3825.580360,0,1,...,0,0,1,0,0,1,0,5098.39,1.0,1
27,0,3162.51,2.0,1,68281.0,2668.775192,2405.928415,2879.474068,0,1,...,0,0,1,0,1,0,0,3162.51,1.0,1
46,1,1670.24,16.0,5,40743.0,1465.592847,2371.632377,2444.952182,0,1,...,0,0,1,0,0,0,1,1670.24,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575915,0,2326.28,10.0,1,65670.0,2462.056261,2075.583470,3108.454800,0,1,...,0,0,1,0,0,0,1,2326.28,1.0,1
575917,0,15007.96,13.0,1,156528.0,14881.988575,11528.804235,8294.507867,0,1,...,0,0,1,0,0,1,0,15007.96,1.0,1
575918,0,2331.16,15.0,1,108514.0,2134.562840,2458.185246,3725.138234,0,1,...,0,0,0,1,0,0,1,2331.16,1.0,1
575920,0,6007.08,18.0,2,270606.0,6286.888017,7776.225273,6776.217049,0,1,...,0,0,1,0,0,0,1,6007.08,1.0,1


In [115]:
import sympy as sp
import numpy as np
import pandas as pd

# Definindo as variáveis simbólicas
P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26 = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26')

# Definindo a função de demanda
demanda = sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26)/(1+sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26))

# Derivada da demanda em relação ao preço
demanda_derivada_P = sp.diff(demanda, P)

# Funções lambdify para avaliação numérica
demanda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26), demanda, 'numpy')
demanda_derivada_P_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26), demanda_derivada_P, 'numpy')


# Supondo que df é um DataFrame do pandas com os dados necessários
#df = df_cat_glm

# Extraindo os valores das variáveis
lista_variaveis_na_ordem = ['is_casco_ofer','GBM_mercado','Premio_IC_100','premio_anterior','MD_tempo_casa_2','per_com','per_com_bin_p_1_10','canal_bin_Corretor_Mais','Cod_End_bin_13509','fator_ajuste_bin_f_101_120','classe_bonus_bin_00','UF_bin_AL_CE_RS','UF_bin_AP_DF_ES_PA_PB_RN_SC','UF_bin_GO_MA_MG','UF_bin_MT_PR','UF_bin_RJ','MD_comb_bin_Gasolina','md_tipo_renov_ant_bin_1_SeguroNovo','md_tipo_renov_ant_bin_3_RenovCongenere','fx_ir_12m_bin_01_00_20','fx_ir_12m_bin_03_40_60','fx_ir_12m_bin_04_60_80','fx_ir_12m_bin_05_80_100','MD_tipo_renov_bin_3_RenovPropria_com_sin','idade_ult_RNS_bin_01_05_anos','idade_ult_RNS_bin_06_inf']

X_vals = np.array(df.loc[:, lista_variaveis_na_ordem])
P_val = df['premio'].values

# Calculando os valores de demanda e sua derivada
D_val = demanda_func(P_val, *X_vals.T)
dD_dP_val = demanda_derivada_P_func(P_val, *X_vals.T)

# Calculando a elasticidade-preço da demanda
elasticidade_preco = (dD_dP_val * P_val) / D_val

# Exibindo os resultados
print("Demanda (D):", D_val)
print("Derivada da Demanda em relação ao Preço (dD/dP):", dD_dP_val)
print("Elasticidade-Preço da Demanda (Ep):", elasticidade_preco)

Demanda (D): [0.19465364 0.2086642  0.14825357 ... 0.43359875 0.49695501 0.46909072]
Derivada da Demanda em relação ao Preço (dD/dP): [-2.22773291e-04 -4.95793244e-05 -8.77693437e-05 ... -3.76587148e-04
 -1.48719220e-04 -3.30806207e-04]
Elasticidade-Preço da Demanda (Ep): [-2.84714156 -2.81004279 -3.01835793 ... -2.02464814 -1.79768438
 -1.90030101]


In [116]:
np.mean(elasticidade_preco)

-1.7591950103137135

In [117]:
np.max(elasticidade_preco)

-0.030124737394791993

In [118]:
np.min(elasticidade_preco)

-3.4937747026576647

In [119]:
df['X27'].mean()

1.0

In [98]:
#df.loc[df['numbers'] > 1, '>1'] = 'True'
#df.loc[df['numbers'] <=1 , '>1'] = 'False'
#df

In [120]:
import sympy as sp
import numpy as np
import pandas as pd

# Definindo as variáveis simbólicas
P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, X27 = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 X27')

# Definindo a função de demanda
demanda = sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26)/(1+sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26 -3*X27))

# Derivada da demanda em relação ao preço
demanda_derivada_P = sp.diff(demanda, P)

# Funções lambdify para avaliação numérica
demanda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda, 'numpy')
demanda_derivada_P_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda_derivada_P, 'numpy')


# Supondo que df é um DataFrame do pandas com os dados necessários
df = df_cat_glm

# Extraindo os valores das variáveis
lista_variaveis_na_ordem = ['is_casco_ofer','GBM_mercado','Premio_IC_100','premio_anterior','MD_tempo_casa_2','per_com','per_com_bin_p_1_10','canal_bin_Corretor_Mais','Cod_End_bin_13509','fator_ajuste_bin_f_101_120','classe_bonus_bin_00','UF_bin_AL_CE_RS','UF_bin_AP_DF_ES_PA_PB_RN_SC','UF_bin_GO_MA_MG','UF_bin_MT_PR','UF_bin_RJ','MD_comb_bin_Gasolina','md_tipo_renov_ant_bin_1_SeguroNovo','md_tipo_renov_ant_bin_3_RenovCongenere','fx_ir_12m_bin_01_00_20','fx_ir_12m_bin_03_40_60','fx_ir_12m_bin_04_60_80','fx_ir_12m_bin_05_80_100','MD_tipo_renov_bin_3_RenovPropria_com_sin','idade_ult_RNS_bin_01_05_anos','idade_ult_RNS_bin_06_inf','X27']

X_vals = np.array(df.loc[:, lista_variaveis_na_ordem])
P_val = df['premio'].values

# Calculando os valores de demanda e sua derivada
D_val = demanda_func(P_val, *X_vals.T)
dD_dP_val = demanda_derivada_P_func(P_val, *X_vals.T)

# Calculando a elasticidade-preço da demanda
elasticidade_preco = (dD_dP_val * P_val) / D_val

# Exibindo os resultados
print("Demanda (D):", D_val)
print("Derivada da Demanda em relação ao Preço (dD/dP):", dD_dP_val)
print("Elasticidade-Preço da Demanda (Ep):", elasticidade_preco)

Demanda (D): [0.23882781 0.26026918 0.17256294 ... 0.7374268  0.94158264 0.84633094]
Derivada da Demanda em relação ao Preço (dD/dP): [-3.35357361e-04 -7.71347737e-05 -1.18912517e-04 ... -1.08924930e-03
 -5.33887608e-04 -1.07681329e-03]
Elasticidade-Preço da Demanda (Ep): [-3.49326412 -3.50499769 -3.51328271 ... -3.44334436 -3.40607978
 -3.42851282]


In [121]:
np.mean(elasticidade_preco)

-3.229573645043233

In [124]:
# Obtenha os coeficientes relevantes
df = df_cat_glm
coef_log_premio = model_glm.params['np.log(premio)']
coef_premio_is_casco_ofer = model_glm.params['np.divide(premio, is_casco_ofer)']
coef_premio_GBM_mercado = model_glm.params['np.log(np.divide(premio, GBM_mercado))']
coef_premio_Premio_IC_100 = model_glm.params['np.log(np.divide(premio, Premio_IC_100))']
coef_premio_anterior = model_glm.params['np.log((np.divide(premio, premio_anterior)))']

# Calcule a previsão do modelo
df['pred_prob'] = model_glm.predict(df)

# Calcule a elasticidade ajustada
df['elasticidade'] = (
    coef_log_premio * (df['premio'] +
    coef_premio_is_casco_ofer * (df['premio'] / df['is_casco_ofer']) +
    coef_premio_GBM_mercado * (df['premio'] / df['GBM_mercado']) +
    coef_premio_Premio_IC_100 * (df['premio'] / df['Premio_IC_100']) +
    coef_premio_anterior * (df['premio'] / df['premio_anterior']) )/np.mean(df['premio'])
)

print(df[['premio', 'pred_prob', 'elasticidade']])

          premio  pred_prob  elasticidade
8        2487.76   0.194704     -0.220160
11      11826.61   0.208735     -1.048174
12       5098.39   0.148306     -0.451662
27       3162.51   0.202198     -0.280018
46       1670.24   0.806874     -0.147844
...          ...        ...           ...
575915   2326.28   0.445986     -0.205952
575917  15007.96   0.056130     -1.330155
575918   2331.16   0.433678     -0.206408
575920   6007.08   0.497059     -0.532316
575921   2694.67   0.469183     -0.238650

[64070 rows x 3 columns]


In [125]:
df["elasticidade"].mean()

-0.25726733452629835

In [61]:
#print(dir(model_glm))

In [62]:
df_cat_glm["prob_est"]=model_glm.predict(df_cat_glm)

In [63]:
df_cat_glm["prob_est"].mean()

0.5029811144061183

## Base Para escorar e otimizar

In [64]:
#df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",

In [65]:
import pandas as pd
import numpy as np

In [66]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Data Table - Otimizacao segmentacao 49.7.csv",
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [67]:
#df_lote_102024.columns.to_list()

In [68]:
np.mean(df_lote_102024['Renewal Demand'])

0.49692578162505185

In [69]:
np.mean(df_lote_102024['Demanda_basecase'])

0.4969852338386432

In [70]:
#df_lote_102024.groupby(['Amostra_2'])['Premio_oferecido_com_batente','Premium Parameter','Mercado_Mediana','GBM_mercado',"Renewal Demand"].describe().T

In [71]:
#df_lote_092024.columns.to_list()

In [72]:
df_lote_escorar=df_lote_102024.copy()

In [73]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [74]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [75]:
df_lote = categorizar(df_lote_escorar)

In [76]:
df_lote.shape

(1459, 145)

In [77]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [78]:
#df_cat_glm2.columns.to_list()

In [79]:
df_lote = df_lote
#df_lote = df
#df_lote = df.query('chave == 695900014103100001')
P2=(df_lote['premio'])
X1=np.array(df_lote['is_casco_ofer'])
X2=np.array(df_lote['GBM_mercado'])
X3=np.array(df_lote['Premio_IC_100'])
X4=np.array(df_lote['premio_anterior'])
X5=np.array(df_lote['MD_tempo_casa_2'])
X6=np.array(df_lote['per_com'])
X7=np.array(df_lote['per_com_bin_p_1_10'])
X8=np.array(df_lote['canal_bin_Corretor_Mais'])
X9=np.array(df_lote['Cod_End_bin_13509'])
X10=np.array(df_lote['fator_ajuste_bin_f_101_120'])
X11=np.array(df_lote['classe_bonus_bin_00'])
X12=np.array(df_lote['UF_bin_AL_CE_RS'])
X13=np.array(df_lote['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df_lote['UF_bin_GO_MA_MG'])
X15=np.array(df_lote['UF_bin_MT_PR'])
X16=np.array(df_lote['UF_bin_RJ'])
X17=np.array(df_lote['MD_comb_bin_Gasolina'])
X18=np.array(df_lote['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df_lote['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df_lote['fx_ir_12m_bin_01_00_20'])
X21=np.array(df_lote['fx_ir_12m_bin_03_40_60'])
X22=np.array(df_lote['fx_ir_12m_bin_04_60_80'])
X23=np.array(df_lote['fx_ir_12m_bin_05_80_100'])
X24=np.array(df_lote['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df_lote['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df_lote['idade_ult_RNS_bin_06_inf'])

def demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P2)  + 1.3647 * np.divide(P2, X1)-0.8782 * np.log(np.divide(P2, X2))-0.795 * np.log(np.divide(P2, X3))-1.6731 * np.log((np.divide(P2, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)



In [80]:
df_lote["Demanda_base"]=demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
df_lote["per_com_pct"] = df_lote['per_com']/100
df_lote["premio_prob"] = df_lote["Demanda_base"]*df_lote["premio"]
df_lote["premio_anterior_prob"] = df_lote["Demanda_base"]*df_lote["premio_anterior"]
df_lote["Act_cost_prob"] = df_lote["Demanda_base"]*df_lote["Act_cost"]                                                                
df_lote["comissao_valor"] = df_lote["per_com_pct"] *df_lote["premio"]
df_lote["comissao_valor_prob"] = df_lote["per_com_pct"] *df_lote["premio_prob"]


In [81]:
df_lote.shape

(1459, 183)

In [82]:
np.mean(df_lote['Demanda_base'])

0.5073676747519267

In [83]:
#df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

In [84]:
apolice_ofertada  = np.round(df_lote.shape[0],0)
apolice_estimada  = np.round(np.sum(df_lote["Demanda_base"]),0)
premio_ofertado   = np.mean(df_lote["premio"])
retencao_estimada = np.mean(df_lote["Demanda_base"])
premio_esperado   = np.sum(df_lote["premio_prob"])/np.sum(df_lote["Demanda_base"])
risco_ofertado    = np.mean(df_lote["Act_cost"])
risco_esperado    = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["Demanda_base"])
sp_ofertado       = np.sum(df_lote["Act_cost"])/np.sum(df_lote["premio"])
sp_esperado       = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["premio_prob"])
comis_ofertado    = np.sum(df_lote["comissao_valor"])/np.sum(df_lote["premio"])
comis_esperado    = np.sum(df_lote["comissao_valor_prob"])/np.sum(df_lote["premio_prob"])

premio_anterior   = np.mean(df_lote["premio_anterior"])
premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Demanda_base"])

ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))
print('premio_anterior: %s' % np.round(premio_anterior,2))
print('premio_anterior_w: %s' % np.round(premio_anterior_w,2))


Apolices Ofertadas: 1459
Apolices Renovadas: 740.0
premio_ofertado: 2993.3
retencao_estimada : 50.74
premio_esperado" : 2388.44
risco_ofertado" : 1726.89
risco_esperado" : 1347.66
sp_ofertado" : 57.69
sp_esperado" : 56.42
ic_ofertado" : 100.78
ic_esperado" : 99.58
premio_anterior: 2863.09
premio_anterior_w: 2634.39


## Otimização Teste 

In [85]:
#print(df_lote_otim['elasticidade'].value_counts(ascending=False))

In [86]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1459, 183)

In [87]:
#df_lote_otim_atu.groupby(["Amostra"])["Demanda_base"].describe().T

In [88]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1459, 183)

In [89]:
print(np.mean(df_lote_otim['Demanda_basecase'])),
print(np.mean(df_lote_otim['Renewal Demand'])),
print(np.mean(df_lote_otim['Demanda_base']))

0.4969852338386432
0.49692578162505185
0.5073676747519267


#########################################

In [90]:
df_relatorio_bc = df_lote_otim.copy()

df_relatorio_bc["per_com_pct"] = df_relatorio_bc['per_com']/100
df_relatorio_bc["premio_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio"]
df_relatorio_bc["premio_anterior_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio_anterior"]
df_relatorio_bc["Act_cost_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["Act_cost"]                                                                
df_relatorio_bc["comissao_valor"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio"]
df_relatorio_bc["comissao_valor_prob"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_bc.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_bc["Demanda_base"]),0)
premio_ofertado   = np.mean(df_relatorio_bc["premio"])
retencao_estimada = np.mean(df_relatorio_bc["Demanda_base"])
premio_esperado   = np.sum(df_relatorio_bc["premio_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
risco_ofertado    = np.mean(df_relatorio_bc["Act_cost"])
risco_esperado    = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
sp_ofertado       = np.sum(df_relatorio_bc["Act_cost"])/np.sum(df_relatorio_bc["premio"])
sp_esperado       = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_bc["comissao_valor"])/np.sum(df_relatorio_bc["premio"])
comis_esperado    = np.sum(df_relatorio_bc["comissao_valor_prob"])/np.sum(df_relatorio_bc["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749


premio_anterior_ofer    = np.mean(df_relatorio_bc["premio_anterior"])
premio_anterior_w   = np.sum(df_relatorio_bc["premio_anterior_prob"])/np.sum(df_relatorio_bc["Demanda_base"])

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))

print('comis_ofertado" : %s'  % np.round(comis_ofertado*100,2))
print('comis_esperado" : %s'  % np.round(comis_esperado*100,2))

print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

print('premio_anterior_ofer" : %s'  %np.round( premio_anterior_ofer,2))
print('premio_anterior_w" : %s'  %np.round( premio_anterior_w,2))

Apolices Ofertadas: 1459
Apolices Renovadas: 740.0
premio_ofertado: 2993.3
retencao_estimada : 50.74
premio_esperado" : 2388.44
risco_ofertado" : 1726.89
risco_esperado" : 1347.66
comis_ofertado" : 15.59
comis_esperado" : 15.66
sp_ofertado" : 57.69
sp_esperado" : 56.42
ic_ofertado" : 100.78
ic_esperado" : 99.58
premio_anterior_ofer" : 2863.09
premio_anterior_w" : 2634.39


In [91]:
#df2 = df[['premio','is_casco_ofer','Cod_End_bin_Demais','Act_cost']]

In [92]:
#df2.to_excel(r'analise_teste.xlsx',  header='true')

In [93]:
#v2=np.mean(df2['is_casco_ofer'])
#indicadora = np.array(df2['Cod_End_bin_Demais']) 
#precos_iniciais = np.array(df2['premio']) 

In [94]:
#[x for x in df.columns if "com_bin" in x]

In [ ]:
df["IC_ofer_python0"] = df["per_com_pct"] + df["Act_cost"]/df["premio"] +  0.2749

for index, row in df.iterrows():
    if row['IC_ofer_python0']   <= 0.8:
        df.at[index, 'fat_ajuste_otim0'] = 0.95
    elif row['IC_ofer_python0']   <= 0.95:
        df.at[index, 'fat_ajuste_otim0'] = 0.92        
    elif row['IC_ofer_python0']   <= 0.99:
        df.at[index, 'fat_ajuste_otim0'] = 0.99

    elif row['IC_ofer_python0']   >= 1.01:
        df.at[index, 'fat_ajuste_otim0'] = 1.02
    elif row['IC_ofer_python0']   >= 1.03:
        df.at[index, 'fat_ajuste_otim0'] = 1.04
    else:
        df.at[index, 'fat_ajuste_otim0'] = 1.0
    

In [95]:
%%time
######
import sympy as sp
from scipy.optimize import fsolve

df = df_lote_otim

target_demand=0.525

fator_retencao = np.mean(df['Demanda_base'])/target_demand

df['fator_retencao']=fator_retencao


def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(P):
    
    #df['precos']= P *df["fat_ajuste_otim0"] 
    df['precos']= P *1.16
    
    
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    # Calcular a margem esperada
    df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'] - df['per_com']/100 - 0.2749*df['precos'])
    
  
    margem_esperada_total = df['margem_esperada_otimizada'].sum() 

    # Maximizar a margem 
    return -margem_esperada_total

# Definir os limites para os preços otimizado
nome_variaveis = ['LB','UB']
limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=12,seed=df.shape[0])

# Resultados
print(resultado.message) 
print("Tamanho da Base:", df.shape[0])
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun


Optimization terminated successfully.
Tamanho da Base: 1459
CPU times: total: 1min 10s
Wall time: 1min 24s


In [96]:
print(df['precos'].mean()),
print(np.mean(precos_otimizados)),
print(np.mean(-resultado.fun))

3658.5962946161403
3009.1527695587147
498203.81187089736


In [ ]:
#df3.to_csv(r'df3_avalia_prem.csv', sep='|' ,header='true')

In [97]:

df['batente_otim_min']=0.9
df['batente_otim_max']=1.1
df['batente_otim_prem_ant_min']=0.5
df['batente_otim_prem_ant_max']=1.8

 
df['precos_otimizados'] = precos_otimizados


df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()


df['fat_teste0']= df['precos_otimizados']/df['premio']
df['fat_teste1']=df['precos_otimizados']/df['premio']-1 

fat_teste0= df['fat_teste0'].values
fat_teste1= df['fat_teste1'].values

if (fat_teste0 > 1).any() ==True:
    fat_teste = fat_teste1
else:
    fat_teste=0
    
df['fat_teste'] =  fat_teste
    
    
delta_q = (df['retencao_otimizada']-df['Demanda_base'])
delta_p = (df['precos_otimizados']-df['premio'])
   
delta_q_b = ((df['retencao_otimizada']+df['Demanda_base'])/2)
delta_p_b = ((df['precos_otimizados']+df['premio'])/2)

df["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
df["elasticidade_2"] = (df['retencao_otimizada']/df['Demanda_base']-1)/(df['precos_otimizados']/df['premio']-1)  

df["elasticidade"] = df["elasticidade"].fillna(-1)
df["elasticidade_2"] = df["elasticidade_2"].fillna(-1)

df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.5,-2.5,-2,-1.5,-1,0])  

df.replace([np.inf, -np.inf], -1, inplace=True)



df["SP_ofer_python"] = df["Act_cost"]/df["precos_otimizados"]
df["IC_ofer_python"] = df["per_com_pct"] + df["SP_ofer_python"] +  0.2749
df["SP_ofer_eranix"] = df["Act_cost"]/df["Premium Parameter"]
df["IC_ofer_eranix"] = df["SP_ofer_eranix"] + df["per_com_pct"]+  0.2749
df["varia_prem_anterior_py"] = df["precos_otimizados"]/df["premio_anterior"]
df["varia_prem_anterior_earnix"] = df["Premium Parameter"]/df["premio_anterior"]
df["varia_prem_ofer_py"] = df["precos_otimizados"]/df["premio"]
df["varia_prem_ofer_earnix"] = df["Premium Parameter"]/df["premio"]
df["varia_prem_py_earnix"] = df["precos_otimizados"]/df["Premium Parameter"]


df['IC_ofer_python_bin'] = pd.cut(x=df['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
df['IC_ofer_eranix'] = pd.cut(x=df['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 

df['fat_ajuste_otim'] = 1.0  

for index, row in df.iterrows():
    if row['IC_ofer_python'] <= 0.95:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.88
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.97
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.015
            
    elif row['IC_ofer_python']<= 0.99:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.99
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.96
            
             
    elif row['IC_ofer_python']<= 1:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1      
            
            
            
    elif row['IC_ofer_python']<= 1.03:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1            
            
    elif row['IC_ofer_python']<= 3:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.02
            
            
#df['fat_ajuste_otim']=fat_ajuste_otim

# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Base Case:", df['Demanda_base'].mean())
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Elasticidade:", df['elasticidade'].mean())
print("Elasticidade2:", df['elasticidade_2'].mean())

Margem Esperada Total Base Case: 770432.568237982
Margem Esperada Total Otimizada: 498203.81187089736
Retenção Média Base Case: 0.5073676747519267
Retenção Média Otimizada: 0.5026309974805719
Premio Medio Otimizado: 3009.1527695587147
Premio Medio Base Case: 2993.303855515689
Elasticidade: -1.6389693210790068
Elasticidade2: -1.6428786657757861


In [ ]:
df['Demanda_base']

In [ ]:
df['precos_otimizados'].mean()

In [ ]:
#df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

In [ ]:
df['fat_teste3']=df['fat_teste2']+df['elasticidade']

In [ ]:
df['fat_teste3'].mean()

In [ ]:
print(df['IC_ofer_python_bin'].value_counts(ascending=False))

In [ ]:
df_t = df.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
#df[['IC_ofer_python','fat_ajuste_otim',"Demanda_base","retencao_otimizada","elasticidade","premio_anterior","premio","precos_otimizados","Premium Parameter"]]

In [ ]:
df['fat_teste'].describe()

In [ ]:
print(df['IC_ofer_eranix'].value_counts(ascending=False))

In [ ]:
df['elasticidade_bin'].unique()

In [ ]:
df.groupby(["elasticidade_bin"])["retencao_otimizada","precos_otimizados"].describe().T

In [ ]:
df_ver.loc[1440:1459,["fat_teste0","fat_teste1","fat_teste",'retencao_otimizada','Demanda_base','precos_otimizados','premio','elasticidade']]

In [ ]:
(df['retencao_otimizada']/df['Demanda_base']-1) /(df['precos_otimizados']/df['premio']-1)

In [ ]:
df[["precos_otimizados","premio"]].describe().T

In [ ]:
df[["varia_prem_anterior","varia_prem_ofer","varia_prem_anterior_earnix","varia_prem_ofer_earnix"]].describe().T

In [ ]:
#ver = df.query('varia_prem_ofer >= 1.1')
#ver

In [ ]:
#df_lote_otim_atu.groupby(["Amostra"])["Demanda_base"].describe().T

In [ ]:
ver[["premio_anterior","premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior","varia_prem_ofer"]]

In [ ]:
#data =df[["IC_ofer_python","elasticidade"]]


In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

In [ ]:
 df['retencao_otimizada'].mean()

#criando a função

In [98]:
df_relatorio_otim = df.copy()

#df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*1
df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*df['fat_ajuste_otim']
#df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*df_relatorio_otim['fator_retencao']*df['fat_ajuste_otim']


df_relatorio_otim['batente_otim']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio'],2)
df_relatorio_otim['batente_otim_prem_ant']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio_anterior'],2)

df_relatorio_otim['retencao_otimizada'] = calcular_retencao(df_relatorio_otim['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)


df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim["premio_prob"]

df_relatorio_otim["premio_anterior_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["premio_anterior"]

comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])

premio_anterior_ofer    = np.mean(df_relatorio_otim["premio_anterior"])
premio_anterior_w   = np.sum(df_relatorio_otim["premio_anterior_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])

df_relatorio_otim['var_prem_earnix_py'] = df_relatorio_otim['Premium Parameter']/df_relatorio_otim['precos_otimizados']

df_relatorio_otim['comis_valor_py'] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim['precos_otimizados'] 
df_relatorio_otim['comis_valor_earnix'] =df_relatorio_otim["per_com_pct"]*df_relatorio_otim['Premium Parameter'] 
df_relatorio_otim['DA_Ofer'] = 0.2749


apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749


print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))

print('comis_ofertado" : %s'  % np.round(comis_ofertado*100,2))
print('comis_esperado" : %s'  % np.round(comis_esperado*100,2))

print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

print('premio_anterior_ofer" : %s'  %np.round( premio_anterior_ofer,2))
print('premio_anterior_w" : %s'  %np.round( premio_anterior_w,2))

Apolices Ofertadas: 1459
Apolices Renovadas: 738.0
premio_ofertado: 2994.92
retencao_estimada : 50.58
premio_esperado" : 2399.01
risco_ofertado" : 1726.89
risco_esperado" : 1350.42
comis_ofertado" : 15.59
comis_esperado" : 15.67
sp_ofertado" : 57.66
sp_esperado" : 56.29
ic_ofertado" : 100.74
ic_esperado" : 99.45
premio_anterior_ofer" : 2863.09
premio_anterior_w" : 2626.48


In [ ]:
df_relatorio_otim['precos_otimizados'].mean()

In [ ]:
df_relatorio_otim['batente_otim_prem_ant'].describe()

In [ ]:
df_relatorio_otim['batente_otim'].describe()

In [ ]:
df_relatorio_otim['var_prem_earnix_py'].describe()

In [ ]:
df_t = df_relatorio_otim.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
df_export_analise_2 =df_relatorio_otim[["chave",'IC_ofer_eranix','IC_ofer_python_bin','elasticidade','elasticidade_bin',"Renovacao_ant","flag_idade_ult_rns",'comis_valor_py','comis_valor_earnix','DA_Ofer',"Act_cost","TARIFA","premio_anterior","per_com_pct","precos_otimizados","Premium Parameter","varia_prem_py_earnix","Demanda_base","retencao_otimizada",'Demanda_basecase',"Renewal Demand","IC_ofer_python","IC_ofer_eranix","varia_prem_anterior_py","varia_prem_ofer_py","varia_prem_anterior_earnix","varia_prem_ofer_earnix","SP_ofer_python","SP_ofer_eranix"]]
#df_export_analise_2.columns.to_list()

In [ ]:
df_t = df_export_analise_2.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"precos_otimizados","Premium Parameter","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
df_export_analise_2.groupby(["elasticidade_bin"])["retencao_otimizada","precos_otimizados"].describe().T

In [ ]:
df_export_analise_2.to_csv(r'df_export_analise_3m.csv', sep='|' ,header='true')

In [ ]:
df.to_csv(r'analitico_202410_v1.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]
, sep='|', encoding='utf-8', header='true'

In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

In [100]:
df_teste = df

In [101]:
delta_q = (df_teste['retencao_otimizada']-df_teste['Demanda_base'])
delta_p = (df_teste['precos_otimizados']-df_teste['premio'])
   
delta_q_b = ((df_teste['retencao_otimizada']+df_teste['Demanda_base'])/2)
delta_p_b = ((df_teste['precos_otimizados']+df_teste['premio'])/2)

df_teste["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
df_teste["elasticidade_2"] = (df_teste['retencao_otimizada']/df_teste['Demanda_base']-1)/(df_teste['precos_otimizados']/df_teste['premio']-1)

df_teste.replace([np.inf, -np.inf], -1, inplace=True)
    
df_teste['elasticidade_bin'] = pd.cut(x=df_teste['elasticidade'], bins=[-3.5,-2.5,-2,-1.5,-1,0])  


df_teste["SP_ofer_python"] = df_teste["Act_cost"]/df_teste["precos_otimizados"]
df_teste["IC_ofer_python"] = df_teste["per_com_pct"]+ df_teste["SP_ofer_python"] +  0.2749
df_teste["SP_ofer_eranix"] = df_teste["Act_cost"]/df_teste["Premium Parameter"]
df_teste["IC_ofer_eranix"] = df_teste["SP_ofer_eranix"] + df_teste["per_com_pct"]+  0.2749
df_teste["varia_prem_anterior_py"] = df_teste["precos_otimizados"]/df_teste["premio_anterior"]
df_teste["varia_prem_anterior_earnix"] = df_teste["Premium Parameter"]/df_teste["premio_anterior"]
df_teste["varia_prem_ofer_py"] = df_teste["precos_otimizados"]/df_teste["premio"]
df_teste["varia_prem_ofer_earnix"] = df_teste["Premium Parameter"]/df_teste["premio"]
df_teste["varia_prem_py_earnix"] = df_teste["precos_otimizados"]/df_teste["Premium Parameter"]

df_teste['IC_ofer_python_bin'] = pd.cut(x=df_teste['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
df_teste['IC_ofer_eranix'] = pd.cut(x=df_teste['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 


In [102]:
df['fat_ajuste_otim'] = 1.0  

for index, row in df.iterrows():
    if row['IC_ofer_python'] <= 0.95:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.96
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.96
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.98
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.97
            
    elif row['IC_ofer_python']<= 0.99:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.976
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.972
            
             
    elif row['IC_ofer_python']<= 1:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.991
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.994
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.999           
            
            
            
    elif row['IC_ofer_python']<= 1.03:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1.014
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1.016
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1.013
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1.015
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.018              
            
    elif row['IC_ofer_python']<= 3:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1.02
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.04 